# Hilti Evaluation wrt GT

In [2]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d 

import pandas as pd
import os
import pprint

from evo.core.trajectory import PoseTrajectory3D
from evo.tools import plot
from evo.core import metrics
from evo.core import sync

from pose_trajectory_evaluation import PoseTrajectoryEvaluation

%reload_ext autoreload
%autoreload 2

In [79]:
def transform_prism_position_to_imu(traj, T_prism_imu):
    n_poses = traj.shape[0]
    for i in range(n_poses):
        t_map_prism = np.array([*traj[i,1:4], 1.0])
        traj[i,1:4] = (t_map_prism @ T_prism_imu)[0:3]
    return traj

T_optitrack_prism = np.array(
    [[1.0, 0.0, 0.0, 0.0],
     [0.0, 1.0, 0.0, 0.0],
     [0.0, 0.0, 1.0, 0.0],
     [0.0, 0.0, 0.0, 1.0]])

T_cam0_optitrack = np.array(
    [[0.9837450,  0.1793231, -0.0094284, 0.05354253273380533],
     [0.0085107,  0.0058861,  0.9999465, -0.2786560903711294],
     [0.1793690, -0.9837726,  0.0042642, -0.057329537886130204],
     [0.0, 0.0, 0.0, 1.0]])

T_imu_cam0 = np.array(
    [[-0.0028215, -0.0030748,  0.9999913, 0.05067834857850693],
     [-0.9999955, -0.0010373, -0.0028247, 0.0458784339890185],
     [0.0010459, -0.9999948, -0.0030718, -0.005943648304780761],
     [0.0, 0.0, 0.0, 1.0]])

# T_map_prism  -> T_map_imu
T_prism_imu = np.linalg.inv(T_imu_cam0 @ T_cam0_optitrack @ T_optitrack_prism)

export_paths = ['/media/berlukas/Data/data/datasets/Hilti/Lab/', '/media/berlukas/Data/data/datasets/Hilti/Basement_1/']
exported_missions = ['53ab6674092fa0161000000000000000', '3dd89f6d25cca0161000000000000000']
prisms = [False, True]
eval = PoseTrajectoryEvaluation()

est_trajectories = []
gt_trajectories = []

assert len(export_paths) == len(exported_missions)
assert len(export_paths) == len(prisms)
for export_path, mission_id, is_prism in zip(export_paths, exported_missions, prisms):
    est_traj_filename = export_path + 'vertex_poses_velocities_biases.csv'
    gt_traj_filename = export_path + 'gt.txt'

    est_df = eval.get_mission_from_csv(est_traj_filename, mission_id)
    est_trajectories.append(eval.convert_df_to_traj(est_df))
    
    gt_df = eval.parse_trajectory_txt(gt_traj_filename)
    gt_traj = eval.convert_df_to_traj(gt_df)
    if is_prism:
        gt_trajectories.append(transform_prism_position_to_imu(gt_traj, T_prism_imu))
    else:
        gt_trajectories.append(gt_traj)
    
assert len(gt_trajectories) == len(est_trajectories)
print(f'Loaded {len(gt_trajectories)} missions.')

Loaded 2 missions.


------------------------------------------------------

In [81]:
max_diff = 0.1

delta = 2
delta_unit = metrics.Unit.frames
all_pairs = False

In [53]:
def plot_evaluations_APE(data, ape_metric, ape_stats):
    # Trajectory plot
    fig = plt.figure(figsize=(8, 6), dpi=160)
    traj_by_label = {
        "estimate": data[1],
        "reference": data[0]
    }
    plot.trajectories(fig, traj_by_label, plot.PlotMode.xy)
    plt.show()

    # Trajectory plot
    seconds_from_start = [t - data[1].timestamps[0] for t in data[1].timestamps]
    fig = plt.figure(figsize=(8, 6), dpi=160)
    plot.error_array(fig.gca(), ape_metric.error, x_array=seconds_from_start,
                     statistics={s:v for s,v in ape_stats.items() if s != "sse"},
                     name="APE", title="APE w.r.t. " + ape_metric.pose_relation.value, xlabel="$t$ (s)")
    plt.show()

    plot_mode = plot.PlotMode.xy
    fig = plt.figure(figsize=(8, 6), dpi=160)
    ax = plot.prepare_axis(fig, plot_mode)
    plot.traj(ax, plot_mode, gt_traj, '--', "gray", "reference")
    plot.traj_colormap(ax, data[1], ape_metric.error, 
                       plot_mode, min_map=ape_stats["min"], max_map=ape_stats["max"])
    ax.legend()
    plt.show()

def perform_evaluation_using_data(data, delta, delta_unit, all_pairs):
    print('--- Translational Part -----------------------------')
    pose_relation = metrics.PoseRelation.translation_part
    print('APE:')
    ape_metric_trans = metrics.APE(pose_relation)
    ape_metric_trans.process_data(data)
    ape_stat_trans = ape_metric_trans.get_statistic(metrics.StatisticsType.rmse)
    ape_stats_trans = ape_metric_trans.get_all_statistics()
    pprint.pprint(ape_stats_trans)

    print('RPE:')
    rpe_metric_trans = metrics.RPE(pose_relation, delta, delta_unit, all_pairs)
    rpe_metric_trans.process_data(data)
    rpe_stat_trans = rpe_metric_trans.get_statistic(metrics.StatisticsType.rmse)
    rpe_stats_trans = rpe_metric_trans.get_all_statistics()
    pprint.pprint(rpe_stats_trans)
    print('\n\n')

    print('--- Rotational Part -----------------------------')
    pose_relation = metrics.PoseRelation.rotation_angle_deg
    print('APE:')
    ape_metric_rot = metrics.APE(pose_relation)
    ape_metric_rot.process_data(data)
    ape_stat_rot = ape_metric_rot.get_statistic(metrics.StatisticsType.rmse)
    ape_stats_rot = ape_metric_rot.get_all_statistics()
    pprint.pprint(ape_stats_rot)

    print('RPE:')
    rpe_metric_rot = metrics.RPE(pose_relation, delta, delta_unit, all_pairs)
    rpe_metric_rot.process_data(data)
    rpe_stat_rot = rpe_metric_rot.get_statistic(metrics.StatisticsType.rmse)
    rpe_stats_rot = rpe_metric_rot.get_all_statistics()
    pprint.pprint(rpe_stats_rot)

    plot_evaluations_APE(data, ape_metric_trans, ape_stats_trans)

In [82]:
def ts_ns_to_seconds(ts_ns):
    k_ns_per_s = 1e9;
    return ts_ns / k_ns_per_s;

def convert_est_to_traj(trajectory):
    ts = ts_ns_to_seconds(trajectory[:,0])
    xyz = trajectory[:,1:4]
    wxyz = trajectory[:,4:8]
    return PoseTrajectory3D(positions_xyz = xyz, orientations_quat_wxyz = wxyz, timestamps = ts)

def convert_gt_to_traj(trajectory):
    ts = trajectory[:,0]
    xyz = trajectory[:,1:4]
    wxyz = trajectory[:,4:8]
    return PoseTrajectory3D(positions_xyz = xyz, orientations_quat_wxyz = wxyz, timestamps = ts)

idx = 1

print(f'=== Maplab ===========================================================')
est_traj = convert_est_to_traj(est_trajectories[idx])
gt_traj = convert_gt_to_traj(gt_trajectories[idx])
gt_traj, est_traj = sync.associate_trajectories(gt_traj, est_traj, max_diff)
est_traj.align(gt_traj, correct_scale=False, correct_only_scale=False, n=4)
# gt_traj.align(est_traj, correct_scale=False, correct_only_scale=False, n=4)

data = (gt_traj, est_traj)
perform_evaluation_using_data(data, delta, delta_unit, all_pairs)

=== Maplab ===========================================================
--- Translational Part -----------------------------
APE:
{'max': 0.21989922061305606,
 'mean': 0.16425392436899508,
 'median': 0.15560486546688224,
 'min': 0.12590674592915976,
 'rmse': 0.16783121472245693,
 'sse': 0.11266926654086178,
 'std': 0.03446686763545379}
RPE:
{'max': 19.60245203783361,
 'mean': 19.60245203783361,
 'median': 19.60245203783361,
 'min': 19.60245203783361,
 'rmse': 19.60245203783361,
 'sse': 384.2561258955671,
 'std': 0.0}



--- Rotational Part -----------------------------
APE:
{'max': 92.60075031844376,
 'mean': 68.06893978278768,
 'median': 61.554073359358654,
 'min': 56.56686209398965,
 'rmse': 69.56019594656478,
 'sse': 19354.483440497945,
 'std': 14.326210139869906}
RPE:
{'max': 30.614294473226394,
 'mean': 30.614294473226394,
 'median': 30.614294473226394,
 'min': 30.614294473226394,
 'rmse': 30.614294473226394,
 'sse': 937.2350260934202,
 'std': 0.0}


/usr/local/lib/python3.6/dist-packages/matplotlib/pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/matplotlib/__init__.py:941: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [67]:
print(gt_trajectories[0][0,0])
print(est_trajectories[0][0,0])

1629728522.01
1.6297285221327173e+18
